# Import

In [95]:
import csv
import os
import pickle

import numpy as np
import pandas as pd
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from sklearn.metrics import adjusted_mutual_info_score
from sklearn.mixture import GaussianMixture
from smart_open import open
from stop_words import get_stop_words
from tqdm import tqdm

In [2]:
tqdm.pandas()

# Read data

In [3]:
newsgroups_df = pd.read_csv("../../../Preprocessing/data/20News/master.csv", index_col=0)

In [82]:
with open("../../../Preprocessing/data/20News/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader][0]

In [5]:
vector_sizes = [2, 3, 4, 6, 7, 8, 9, 10, 20, 40, 80, 160, 320, 640]

In [6]:
class Corpus:
    def __init__(self, path=None, texts=None):
        self.path = path
        self.texts = texts
        self.dictionary = Dictionary(texts)

    def __iter__(self):
        if self.path is not None:
            for line in open(self.path):
                # assume there's one document per line, tokens separated by whitespace
                yield self.dictionary.doc2bow(line.lower().split())
        else:
            for line in self.texts:
                yield self.dictionary.doc2bow(line)

In [54]:
texts = newsgroups_df.words_nonstop.progress_apply(lambda x: x.split(' ')).tolist()
corpus = Corpus(texts=texts)
dictionary = Dictionary(texts)

100%|██████████| 18846/18846 [00:01<00:00, 14453.26it/s]


In [78]:
label = newsgroups_df["class"].to_numpy()

In [56]:
model_nums = 2
vector_model_num = 0

In [100]:
stop_words_add = ["would", "could", "should"]
stop_char = ["==", "--", "\'s", "''", "n't", "``","..", "...", "....", "'m", "'ve", "'d", "'ll", ""]
stop_words = stop_words + stop_words_add + stop_char

In [109]:
dictionaries = []
for class_label in tqdm(class_labels):
    localized_df = newsgroups_df[newsgroups_df["class"]==class_label]
    texts = localized_df.words_nonstop.apply(lambda x: x.split(' ')).tolist()
#     一文字以下の単語とstop_word, stop_charを削除
    texts = [[word for word in text if len(word) > 1 if word not in stop_words] for text in texts]
    corpus = Corpus(texts=texts)
    dictionary = Dictionary(texts)
#     登場頻度80%以上の単語を削除
    dictionary.filter_extremes(no_above=0.8)
    dictionaries.append(dictionary)

100%|██████████| 20/20 [01:17<00:00,  3.89s/it]


In [108]:
for i, dictionary in enumerate(dictionaries):
    with open(f'./{i}.txt', mode="w+", newline='') as f:
        write = csv.writer(f)
        write.writerows(dictionary.most_common())